1.read data from the sales_samples.csv file and analyse to identify problems


In [0]:
file_schema ="""
id int,
name string,
dop string,
phone long,
amount string,
discount string
"""

In [0]:
sales_raw_df =(
    spark.read.format("csv")
    .option("header","true")
    .schema(file_schema)
    .load("/Volumes/dev/spark_db/datasets/spark_programming/data/sales_sample.csv")
)

In [0]:
sales_raw_df.display()